In [9]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

In [27]:
df=pd.read_excel('data/Keywords.xlsx',sheet_name="Wilmar International")

In [28]:
df

,link,headline
0,https://www.amnesty.org/en/latest/news/2016/11...,www.amnesty.org › en › latestPalm Oil: Global ...
1,https://www.amnesty.org/en/wp-content/uploads/...,www.amnesty.org › en › wp-contentAMNESTY INTER...
2,https://news.mongabay.com/2023/12/a-decade-of-...,news.mongabay.com › 2023 › 12A decade of stopp...
3,https://www.wilmar-international.com/docs/defa...,www.wilmar-international.com › docs › default-...
4,https://www.eco-business.com/news/amnesty-inte...,www.eco-business.com › news › amnesty-internat...
...,...,...
970,https://e-recruitment.wilmar.co.id/,e-recruitment.wilmar.co.idHome Page - E-Recrui...
971,https://www.ingredientsnetwork.com/wilmar-inte...,www.ingredientsnetwork.com › wilmar-internatio...
972,https://www.infrontanalytics.com/fe-EN/30304FN...,www.infrontanalytics.com › fe-EN › 30304FNWilm...
973,https://www.marketscreener.com/quote/stock/WIL...,www.marketscreener.com › quote › stockWilmar I...


In [50]:
df1=df[0:4]

In [51]:
df1

,link,headline,article_content
0,https://www.amnesty.org/en/latest/news/2016/11...,www.amnesty.org › en › latestPalm Oil: Global ...,
1,https://www.amnesty.org/en/wp-content/uploads/...,www.amnesty.org › en › wp-contentAMNESTY INTER...,None
2,https://news.mongabay.com/2023/12/a-decade-of-...,news.mongabay.com › 2023 › 12A decade of stopp...,
3,https://www.wilmar-international.com/docs/defa...,www.wilmar-international.com › docs › default-...,NaN


In [7]:
import requests

def is_pdf(url):
    # Send a HEAD request to get the response headers
    response = requests.head(url)
    
    # Get the content type from the response headers
    content_type = response.headers.get('Content-Type', '')
    
    # Check if 'pdf' is present in the content type
    return 'pdf' in content_type.lower()

def is_pdf_or_website(url):
    # Check if the URL ends with '.pdf'
    if url.lower().endswith('.pdf'):
        return True
    return is_pdf(url)

# Test the function
url1 = "https://www.wilmar-international.com/docs/default-source/default-document-library/sustainability/grievance/asa2151842016english_the-great-palm-oil-scandal.pdf?sfvrsn=d4630ec8_2"
url2 = "https://www.wilmar-international.com"

print(is_pdf_or_website(url1))  # Output: True
print(is_pdf_or_website(url2))  # Output: False


True
False


In [58]:
from urllib import request

def download_pdf(url: str, pdf_file_name: str) -> str:
    """Download PDF from given URL to local directory using proxy."""
    try:


        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        req = request.Request(url, headers=headers)
        with request.urlopen(req) as response:
            if response.status == 200:
                current_path=f'{os.getcwd()}/pdf/'
                filepath = os.path.join(current_path, pdf_file_name)
                with open(filepath, 'wb') as pdf_object:
                    pdf_object.write(response.read())
                print(f'{pdf_file_name} was successfully saved!')
                return filepath  # Return the path to the downloaded PDF file
            else:
                print(f'Uh oh! Could not download {pdf_file_name}, HTTP response status code: {response.status}')
    except Exception as e:
        print(f"An error occurred: {str(e)}")

 

In [59]:
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader

def scrape_links(df, pdf_dir='pdf/', json_dir='json/'):
    os.makedirs(pdf_dir, exist_ok=True)
    os.makedirs(json_dir, exist_ok=True)
    
    for index, row in df.iterrows():
        link = row['link']
        file_name = link.split('/')[-1]  # Extract file name from the link
        
        if link.endswith('.pdf'):

            pdf_source=download_pdf(link,file_name)

            
            # Update DataFrame with source of PDF
            df.at[index, 'source'] = 'pdf'
            df.at[index, 'source_path'] = pdf_source
            
        else:
            try:
                response = requests.get(link)
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract name and keywords
                name = soup.title.string.strip()
                keywords = [word.strip() for word in name.split()]

                # Initialize content dictionary
                content = {
                    "name": name,
                    "keywords": keywords,
                    "url": link,
                    "content": {
                        "headings": [],
                        "span_tags": [],
                        "p_tags": []
                    }
                }

                # Extract headings, articles, and paragraphs
                headings = soup.find_all(['h1', 'h2', 'h3'])
                for heading in headings:
                    heading_content = heading.get_text().strip()
                    articles = []
                    article = {
                        "content": "",
                        "paragraphs": []
                    }
                    next_element = heading.find_next_sibling()
                    while next_element and next_element.name not in ['h1', 'h2', 'h3']:
                        if next_element.name == 'p':
                            article["paragraphs"].append({"content": next_element.get_text().strip()})
                        elif next_element.name in ['h1', 'h2', 'h3']:
                            break
                        else:
                            article["content"] += next_element.get_text().strip() + " "
                        next_element = next_element.find_next_sibling()
                    if article["content"] or article["paragraphs"]:
                        articles.append(article)
                    content["content"]["headings"].append({
                        "tag": heading.name,
                        "content": heading_content,
                        "articles": articles
                    })

                # Extract span tags
                spans = soup.find_all('span')
                for span in spans:
                    content["content"]["span_tags"].append({"content": span.get_text().strip()})

                # Extract p tags
                paragraphs = soup.find_all('p')
                for paragraph in paragraphs:
                    content["content"]["p_tags"].append({"content": paragraph.get_text().strip()})

                # Save scraped data to JSON
                json_path = os.path.join(json_dir, f'{file_name}.json')
                with open(json_path, 'w') as json_file:
                    json.dump({"data": content}, json_file)

                # Update DataFrame with source of JSON data
                df.at[index, 'source'] = 'json'
                df.at[index, 'source_path'] = json_path

            except Exception as e:
                print(f"Error processing link {link}: {e}")
    
    return df

# Example usage:
# df = scrape_links(df)


In [60]:
# Example usage:
df3 = scrape_links(df1)


ASA2158192017ENGLISH.pdf was successfully saved!


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Error processing link https://www.wilmar-international.com/docs/default-source/default-document-library/sustainability/grievance/asa2151842016english_the-great-palm-oil-scandal.pdf?sfvrsn=d4630ec8_2: 'NoneType' object has no attribute 'string'


In [47]:
import os
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
import PyPDF2
from urllib import request

       
def process_link(row):
    link = row['link']
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    if link.endswith('.pdf'):
        # Step 1: If link points to a PDF, download and save it
        file_name = os.path.basename(link)
        pdf_dir = 'pdf'
        os.makedirs(pdf_dir, exist_ok=True)
        path_link=download_pdf(link, file_name)

        return path_link
    else:
        # Step 2: If not a PDF, scrape article from webpage
        response = rq.get(link,headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find article content
        article_content = soup.find_all('p')
        # Extract text
        article_text = ' '.join([para.get_text(strip=True) for para in article_content])
        return article_text





In [49]:

df1['article_content'] = df1.apply(process_link, axis=1)

# Save DataFrame to CSV
df.to_csv('articles.csv', index=False)

print(df)

ASA2158192017ENGLISH.pdf was successfully saved!
                                                  link  \
0    https://www.amnesty.org/en/latest/news/2016/11...   
1    https://www.amnesty.org/en/wp-content/uploads/...   
2    https://news.mongabay.com/2023/12/a-decade-of-...   
3    https://www.wilmar-international.com/docs/defa...   
4    https://www.eco-business.com/news/amnesty-inte...   
..                                                 ...   
970                https://e-recruitment.wilmar.co.id/   
971  https://www.ingredientsnetwork.com/wilmar-inte...   
972  https://www.infrontanalytics.com/fe-EN/30304FN...   
973  https://www.marketscreener.com/quote/stock/WIL...   
974  https://sginvestors.io/analysts/research/2023/...   

                                              headline article_content  
0    www.amnesty.org › en › latestPalm Oil: Global ...                  
1    www.amnesty.org › en › wp-contentAMNESTY INTER...            None  
2    news.mongabay.com › 2023 › 12A

C:\Users\faiza\AppData\Local\Temp\ipykernel_4504\1688855881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['article_content'] = df1.apply(process_link, axis=1)


In [14]:
df1 = scrape_links(df)

C:\Users\faiza\AppData\Local\Temp\ipykernel_4504\3838066016.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'source'] = 'json'
C:\Users\faiza\AppData\Local\Temp\ipykernel_4504\3838066016.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'source_path'] = json_path
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Error processing link https://www.wilmar-international.com/docs/default-source/default-document-library/sustainability/grievance/asa2151842016english_the-great-palm-oil-scandal.pdf?sfvrsn=d4630ec8_2: 'NoneType' object has no attribute 'string'


In [46]:
import urllib.request
with urllib.request.urlopen('https://www.amnesty.org/en/latest/news/2016/11/palm-oil-global-brands-profiting-from-child-and-forced-labour/') as response:
   html = response.read()
   print(html)

HTTPError: HTTP Error 403: Forbidden

In [ ]:
{
"data":{
    "name":"Wilmar International",
    "keywords":[
        "Wilmar",
        "International",
        "Wilmar International"
    ],
    "url":"https://www.wilmarinternational.com/",
    "content":{
        "heading tag":{
            "identifier":"identify the contain",
            "content":"content of headings"
        },
        "article tag":{
            "content":"content of articles"
        },
        "span tag":{
            "content":"content of span"
        },
        "p tag":{
            "content":""
        },

    }
    
}
}

In [8]:
df['link']

0      https://www.amnesty.org/en/latest/news/2016/11...
1      https://www.amnesty.org/en/wp-content/uploads/...
2      https://news.mongabay.com/2023/12/a-decade-of-...
3      https://www.wilmar-international.com/docs/defa...
4      https://www.eco-business.com/news/amnesty-inte...
                             ...                        
970                  https://e-recruitment.wilmar.co.id/
971    https://www.ingredientsnetwork.com/wilmar-inte...
972    https://www.infrontanalytics.com/fe-EN/30304FN...
973    https://www.marketscreener.com/quote/stock/WIL...
974    https://sginvestors.io/analysts/research/2023/...
Name: link, Length: 975, dtype: object